In [1]:
import datetime, pandas as pd, requests, csv, os
import urllib.request, json
from os import listdir
from os.path import isfile, join
import os
current_path = os.getcwd()

C:\Users\foyke\AppData\Local\Temp\ipykernel_40412\162770284.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import datetime, pandas as pd, requests, csv, os


In [13]:
divs_all = []

In [14]:
def div_loader(isin):
    global divs_all

    ## Выгрузка по ISIN всех возможных secid, к которым привязаны дивиденды
    # isin = "RU0009029540"
    query  = "https://iss.moex.com/iss/securities.json?q={}&iss.meta=off".format(isin)
    with urllib.request.urlopen(query) as url:
        data = json.load(url)

    # print(data)

    secid_list = []
    for i in range(0, len(data['securities']['data'])):
        secid = data['securities']['data'][i][1]
        secid_list.append(secid)

    # print(secid_list)

    ## Выгрузка дивидендов по каждому secid
    isin_div = []

    for i in range(0, len(secid_list)):
        query = "https://iss.moex.com/iss/securities/{}/dividends.json".format(secid_list[i])
        with urllib.request.urlopen(query) as url:
            data = json.load(url)
        # print(data)

        for j in range (0,len(data['dividends']['data'])):
            tmp = []
            date = data['dividends']['data'][j][2]
            cash = data['dividends']['data'][j][3]
            currency = data['dividends']['data'][j][4]
            # print("{}, {}, {}".format(date, cash, currency))
            tmp.append(isin)
            tmp.append(date)
            tmp.append(cash)
            tmp.append(currency)
            # isin_div.append(tmp)
            divs_all.append(tmp)

    return isin_div

In [3]:
path = "datasets/ticker_lists/moex_full.xlsx"
df = pd.read_excel(path)

In [4]:
df_isin = df['ISIN']
df_isin = df_isin.dropna(how='all')
df_isin.drop_duplicates(keep='first', inplace=True)
df_isin.reset_index(drop=True, inplace=True)

In [16]:
divs_all = []
divs_all_1 = []
for i in range(0, len(df_isin)):
    isin = df_isin[i]
    isin_div = div_loader(isin)
    # if len(isin_div) > 0: divs_all_1.append(isin_div)
    
print(len(divs_all))
# print(len(divs_all_1))

642
0


In [20]:
df_divs_all = pd.DataFrame(divs_all, columns=['ISIN','dt','value','currency'])
# df_divs_all.head(100)

,ISIN,dt,value,currency
0,RU000A0JUG31,2019-06-07,0.11,RUB
1,US29760G1031,2017-11-17,0.04,USD
2,US6708662019,2018-01-25,0.12,USD
3,US7496552057,2016-05-27,0.58,USD
4,US7496552057,2016-09-23,0.23,USD
...,...,...,...,...
95,RU000A0JPGA0,2014-07-04,20.00,RUB
96,RU000A0JPGA0,2014-12-16,25.00,RUB
97,RU000A0JPGA0,2015-06-29,27.00,RUB
98,RU000A0JPGA0,2016-07-01,20.00,RUB


In [23]:
path = current_path + "/datasets/dividends/" + "all"
if len(df_divs_all) > 0: df_divs_all.to_excel(path + ".xlsx",index = False)
if len(df_divs_all) > 0: df_divs_all.to_csv(path + ".csv",index = False)